In [1]:
import os
import matplotlib.pyplot as plt
import h5py
import numpy as np
import yaml
from networkx import random_regular_graph
from quantum_annealing.src.graph_ops import shortest_cycles_distr

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Ariel",
    "font.size": 16,
})

script_dir = os.getcwd()
# ---------- here is the path to the results directory ----------------------
# here is the path to the results directory
# ---------------------------------------------------------------------------
results_dir_path = f"{script_dir}/outputs/qbp/very_large/2024-05-18/11-00-16"
results_path = f"{results_dir_path}/result"
config_path = f"{results_dir_path}/.hydra/config.yaml"
with open(config_path, "r") as f:
    raw_config = f.read()
    config = yaml.safe_load(raw_config)
with h5py.File(results_path, "r") as f:
    pairs = np.array(f["coupled_spin_pairs"])
cycles_distr_our_graph = shortest_cycles_distr(pairs)
cycles_dists_three_regular_ensemble = [shortest_cycles_distr(random_regular_graph(3, 1000, i).edges()) for i in range(100)]
cycles_dists_four_regular_ensemble = [shortest_cycles_distr(random_regular_graph(4, 1000, i).edges()) for i in range(100)]

In [ ]:
from typing import List, Tuple
from math import sqrt

def _extend_list(lst: List, size: int) -> List:
    return lst + [0.] * (size - len(lst))

def _aggregate_stat(
    cycles_distr_ensemble: List[List[int]]
) -> Tuple[List[float], List[float]]:
    if cycles_distr_ensemble:
        cycles_distr = cycles_distr_ensemble[0]
        m1_rest, m2_rest = _aggregate_stat(cycles_distr_ensemble[1:])
        assert len(m1_rest) == len(m2_rest)
        max_len = max(len(cycles_distr), len(m1_rest))
        cycles_distr = _extend_list(cycles_distr, max_len)
        m1_rest = _extend_list(m1_rest, max_len)
        m2_rest = _extend_list(m2_rest, max_len)
        return [a + b for a, b in zip(m1_rest, cycles_distr)], [a + (b ** 2) for a, b in zip(m2_rest, cycles_distr)]
    else:
        return [], []

def get_mean_and_std_of_loops_distr(
    cycles_distr_ensemble: List[List[int]]
) -> Tuple[List[float], List[float]]:
    ensemble_size = len(cycles_distr_ensemble)
    m1, m2 = _aggregate_stat(cycles_distr_ensemble)
    mean = list(map(lambda x: x / ensemble_size, m1))
    std = list(map(lambda args: sqrt((args[0] / ensemble_size - args[1] ** 2)), zip(m2, mean)))
    return mean, std

fig = plt.figure()
x = list(range(1, len(cycles_distr_our_graph) + 1))
plt.plot(x, cycles_distr_our_graph, color="blue", label="Our 3-regular")
mean, std = get_mean_and_std_of_loops_distr(cycles_dists_three_regular_ensemble)
x = list(range(1, len(mean) + 1))
plt.xticks(x)
plt.fill_between(
    x,
    list(map(lambda args: args[0] - args[1], zip(mean, std))),
    list(map(lambda args: args[0] + args[1], zip(mean, std))),
    alpha = 0.2,
    color="blue",
)
plt.plot(x, mean, "--", color="blue", label="Mean 3-regular")
mean, std = get_mean_and_std_of_loops_distr(cycles_dists_four_regular_ensemble)
x = list(range(1, len(mean) + 1))
plt.fill_between(
    x,
    list(map(lambda args: args[0] - args[1], zip(mean, std))),
    list(map(lambda args: args[0] + args[1], zip(mean, std))),
    alpha = 0.2,
    color="red",
)
plt.plot(x, mean, "--", color="red", label="Mean 4-regular")
plt.plot(list(range(1, 14)), 11 * [0] + [1] + [0], color="black", label="IBM heavy-hex")
plt.xlabel("Loop length")
plt.ylabel("Fraction of total loops number")
plt.legend(frameon=False)
fig.savefig(f"{results_dir_path}/loops_distribution.pdf",  bbox_inches="tight")